In [1]:
include("../src/Julia.jl")

parseData (generic function with 1 method)

In [11]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [12]:
graphp = "../../graphs/rand_1000_iu1000/"

"../../graphs/rand_1000_iu1000/"

In [13]:
a = readFromFile(graphp * "graph.mtx");

In [14]:
tree = []
for i in 1:5
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [15]:
x = readFromFile(graphp * "x.vec");

In [21]:
numIts = 200;

@time for i in 1:5
    # a run on standard Float64 data types
    b = lap(a) * (x - mean(x))
    
    @time F = treeSolver(tree[i])
    @time myx64,dbg = pcgV(lap(a), b - mean(b), F, x - mean(x), maxits=numIts, verbose=false);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        aHP = toHighPrecision(a; precision=hp)
        laHP = lap(aHP)
        
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        xHP = xHP - mean(xHP)
        
        bHP = laHP * xHP
        bHP = bHP - mean(bHP)
        
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.002897 seconds (5.14 k allocations: 586.766 KB)
  0.202602 seconds (3.62 M allocations: 91.235 MB, 8.24% gc time)
Finished iteration 1 with errors 0.9959808781747878 0.9988012527286464
Finished iteration 11 with errors 0.6223850259790747 0.653779053634051
Finished iteration 21 with errors 0.2188650771750231 0.20513601629897332
Finished iteration 31 with errors 0.09542388575674987 0.08572781949916111
Finished iteration 41 with errors 0.028080830079768068 0.023522588938973577
Finished iteration 51 with errors 0.007252387341013847 0.005768747944563975
Finished iteration 61 with errors 0.001926788483545438 0.0013744450980570982
Finished iteration 71 with errors 0.0005935116962163527 0.00039752924105361764
Finished iteration 81 with errors 0.00023066946854061152 0.00015598426013791538
Finished iteration 91 with errors 6.459818337570036e-5 4.4532260522855696e-5
Finished iteration 101 with errors 1.9212565203438256e-5 1.3175440608934976e-5
Finished iteration 111 with errors 5.181715398583